In [13]:
import os
import sys 
os.chdir("/workspaces/dev/modules")
sys.path.append("/workspaces/dev/models/PillNet")
os.environ["TF_XLA_FLAGS"] = "--tf_xla_auto_jit=2 --tf_xla_cpu_global_jit=false"
os.environ["XLA_FLAGS"] = "--xla_cpu_multi_thread_eigen=false intra_op_parallelism_threads=1 inter_op_parallelism_threads=1"

In [14]:
from tensorflow import keras
import tensorflow as tf
from datetime import datetime

In [15]:
from Pills.MaskedSequence import MaskedSequence
from PillNetPT import PillNetPT

In [16]:
SOURCE_PATH = '/workspaces/dev/_Shared/Datasets/pills/data'
LABEL_PATH = '/workspaces/dev/_Shared/Datasets/pills/class_label.csv'
SHAPE_PATH = '/workspaces/dev/_Shared/Datasets/pills/class_shape_id.csv'
SHAPE_ID_PATH = '/workspaces/dev/_Shared/Datasets/pills/id_shape.csv'
BACKGROUND_IMAGEES_PATH = '/workspaces/dev/data/pills/background'

In [17]:
TRAIN_DATA_SIZE = 10
VALIDATION_DATA_SIZE = 1
BATCH_SIZE = 8
INPUT_SIZE = (256, 256, 3)

In [18]:
model = PillNetPT()
# model.compile(jit_compile=False)
model.compile(jit_compile=False, run_eagerly=True)
model.build(BATCH_SIZE, INPUT_SIZE)
model.summary()

Model: "pill_net_pt_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ stem (Sequential)               │ (None, 256, 256, 32)   │         4,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ C1 (Sequential)                 │ (None, 256, 256, 32)   │        37,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ C2 (Sequential)                 │ (None, 128, 128, 64)   │       148,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ C3 (Sequential)                 │ (None, 64, 64, 128)    │     1,184,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ P3_conv (Conv2D)                │ (None, 64, 64, 64)     │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ P3_upsample (UpSampling2D)      │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ P2_conv (Conv2D)                │ (None, 128, 128, 32)   │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ P2_upsample (UpSampling2D)      │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ final_conv (Conv2D)             │ (None, 256, 256, 3)    │           867 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,152,779 (15.84 MB)

 Trainable params: 1,383,171 (5.28 MB)

 Non-trainable params: 3,264 (12.75 KB)

 Optimizer params: 2,766,344 (10.55 MB)

In [ ]:
train_sequence = MaskedSequence(
  TRAIN_DATA_SIZE, BATCH_SIZE, input_shape = INPUT_SIZE[:2],
  material_path=SOURCE_PATH, label_csv_path=LABEL_PATH, 
  shape_csv_path=SHAPE_PATH, id_shape_csv_path=SHAPE_ID_PATH,
  background_images_path=BACKGROUND_IMAGEES_PATH,
  use_multiprocessing=True, workers = 12
)

In [ ]:
validation_sequence = MaskedSequence(
  TRAIN_DATA_SIZE, BATCH_SIZE, input_shape = INPUT_SIZE[:2],
  material_path=SOURCE_PATH, label_csv_path=LABEL_PATH, 
  shape_csv_path=SHAPE_PATH, id_shape_csv_path=SHAPE_ID_PATH,
  background_images_path=BACKGROUND_IMAGEES_PATH,
  use_multiprocessing=True, workers = 8
)

In [ ]:
train_dataset = tf.data.Dataset.from_generator(
  lambda: train_sequence, 
  output_signature=(
    tf.TensorSpec(shape=(BATCH_SIZE, *INPUT_SIZE), dtype=tf.float32),
    tf.TensorSpec(shape=(BATCH_SIZE, *INPUT_SIZE), dtype=tf.float32),
  )
)

In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint(
  filepath=f"/workspaces/dev/models/PillNet/checkpoints/PillNetPT_{datetime.now().strftime('%Y_%m_%d')}.keras",
  monitor="feature_matching_metric",
  mode = 'min',
  save_best_only=True,
  save_weights_only=False,
  verbose = 1
)

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(
  monitor="feature_matching_metric",
  mode = 'min',
  patience=10,
  restore_best_weights=True,
  verbose = 1
)

In [ ]:
# keras.config.disable_traceback_filtering()
model.fit(
  train_sequence,
  validation_data=validation_sequence,
  epochs=1,
  callbacks=[checkpoint_cb, early_stopping_cb],
  verbose=1
)

I0000 00:00:1742496845.607045  109603 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-03-20 18:54:17.562404: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:497] Allocator (GPU_0_bfc) ran out of memory trying to allocate 64.00MiB (rounded to 67108864)requested by op Relu
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-03-20 18:54:17.562472: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1053] BFCAllocator dump for GPU_0_bfc
2025-03-20 18:54:17.562483: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1060] Bin (256): 	Total Chunks: 267, Chunks in use: 267. 66.8KiB allocated for chunks. 66.8KiB in use in bin. 42.9KiB client-requested in use in bin.
2025-03-20 18:54:17.562486: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1060] Bin (512): 	Total Chunks: 162, Chunks in use: 162. 81.0KiB allocate

ResourceExhaustedError: Exception encountered when calling ReLU.call().

[1m{{function_node __wrapped__Relu_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Relu] name: [0m

Arguments received by ReLU.call():
  • inputs=tf.Tensor(shape=(16, 128, 128, 64), dtype=float32)

In [ ]:
raise Exception("End")

Exception: End

In [ ]:
model.save(f"/workspaces/dev/models/PillNet/checkpoints/PillNetPT_sub_{datetime.now().strftime('%Y_%m_%d_sub')}.keras")